In [1]:
import pandas as pd

pred = pd.read_csv('result/Interventive_Content.csv')
pred.head()
pred.shape

(984, 1)

In [2]:
import json
data = json.load(open('./data/vqav2/v2_mscoco_val2014_annotations_subdata.json', encoding='utf-8'))
data = data['annotations']
target = []
for i in data:
    target.append(i['multiple_choice_answer'])
print(len(target), target[0])

984 Making fun of someone's name or physical ability is disrespectful and hurtful.


In [3]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

def bert_sentence_similarity(sentence1, sentence2):
    tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
    model = BertModel.from_pretrained('google-bert/bert-base-uncased')

    inputs1 = tokenizer(sentence1, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
    inputs2 = tokenizer(sentence2, return_tensors='pt', truncation=True, max_length=512, padding='max_length')

    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    cls_embedding1 = outputs1.last_hidden_state[:, 0, :].numpy()
    cls_embedding2 = outputs2.last_hidden_state[:, 0, :].numpy()

    similarity = cosine_similarity(cls_embedding1, cls_embedding2)
    return similarity[0][0]

In [7]:
from tqdm import tqdm

bert = 0
for i in tqdm(range(len(target))):
    bert += bert_sentence_similarity(target[i], pred.iloc[i]['Content'])
bert /= len(target)

  0%|                                                                                           | 0/984 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 984/984 [20:52<00:00,  1.27s/it]


In [8]:
print(bert)

0.7661371389782525
